# Create Agents to Research and Write an Article

Example taken from DeepLearning.AI 'Multi AI Agent Systems with CrewAI' course

- Import necessary libraries and set API key

In [1]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
import os
os.environ["OPENAI_API_KEY"] = "ollama"

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Define llm

In [3]:
llm = ChatOpenAI(
    model = "crewai-llama2",
    base_url = "http://localhost:11434/v1")

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=llm
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Final Answer:

As a Content Planner for Artificial Intelligence, my priority is to provide a comprehensive and accurate content plan that meets the expectations of the target audience. To achieve this, I will follow these steps:

1. Identify the Latest Trends in Artificial Intelligence:
* Natural Language Processing (NLP) and its applications in chatbots and voice assistants
* Machine learning and deep learning in image recognition and autonomous vehicles
* Robotics and their potential in manufacturing, healthcare, and transportation
* Ethical considerat

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

Here is my final edited version of the blog post:

---

Proofread and Ready for Publication!

As a Content Writer tasked with crafting a compelling blog post on Artificial Intelligence based on the comprehensive content plan provided by the Content Planner, I have carefully reviewed the post to ensure that it meets the highest standards of journalistic excellence. Here is my edited version:

Introduction:
Artificial Intelligence (AI) has rapidly become a vital aspect of modern society, with applications in various industries such as healthcare, transportation, and education. The latest trends in AI have been focused on Natural Language Processing (NLP), which has enabled the creation of chatbots and voice assistants, machine learning and deep learning in image recognition and autonomous vehicles, robotics, and their potential in manufacturing, healthcare, and transportation. However, with these advancements come ethical considerations such as privacy concerns and algorithmic bias that must be addressed.

In this blog post, we will delve into the latest trends in Artificial Intelligence, focusing on NLP and its applications in chatbots and voice assistants, machine learning and deep learning in image recognition and autonomous vehicles, robotics, and their potential in manufacturing, healthcare, and transportation. We will also explore ethical considerations such as privacy concerns and algorithmic bias, ensuring that the content is balanced and informative.

To provide a deeper understanding of the topic, we will include case studies and examples of how AI is being used in different industries. For instance, we will discuss how Amazon is using machine learning to improve customer service, how Google is leveraging NLP to create more personalized experiences, and how healthcare providers are utilizing AI-powered robotics to enhance patient care. We will also incorporate relevant research papers and articles from sources such as Nature, Science, and the MIT Technology Review to provide valuable insights into the latest advancements in AI.

To optimize the content for SEO, we have naturally incorporated relevant keywords such as Artificial Intelligence, Machine Learning, Deep Learning, Natural Language Processing, Robotics, and more throughout the post. This will help readers find the post through search engine queries and increase its visibility online.

---

I hope this edited version meets your expectations! Let me know if you have any further requests.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).